Colin Wessels 
May 8 2024

Goals:

1) Create rudimentary spectral energy distribution plots directly from the photometric data in the ZFOURGE survey failed galaxies (wavelength on x-axis, flux/luminosity on the y-axis). Similar axes to the output of Cigale.
2) Place each SED plot into a FITS data cube using the spectral_cube library. Simply order the failed SED plots by ZFOURGE ID in the cube and keep the same x and y axes, or possibly use galactic coordinates on x and y axes with wavelength on the z axis and represent intensity with colour/brightness.

Note: Negative fluxes will not be shown on log-log plots,
      Negative error values are set to NaN (appears to only occur when fluxes are strongly negative, caused by some error?)

"Central wavelengths" for each filter are found by selecting the wavelength with maximum transmission. However, the maximum transmission can be quite skewed (far from the center) on many filters. What other methods can be used to find a central value of the wavelength for skewed distributions?

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import ascii
from astropy.table import Table

#import astropy.units as u
#from astropy.io import fits

In [2]:
# Declarations 
data_file = "../ZFOURGE/data/failed_cdfs_table" # This file contains photometric data for all 555 failed ZFOURGE CDFS galaxies
filters_dir = "../ZFOURGE/data/Cowley CIGALE Filters/" # This folder contains all filter transmission tables, used to find central wavelength of filters in table
filter_match = "../ZFOURGE/data/filter_match_table" # An ASCII table with the ZFOURGE filter name in 1st collumn, and filter transmission file name in 2nd col.
output_dir = "../ZFOURGE/outputs/plots/failed_sed_plots/" # Directory to output all galaxy plots

In [11]:
# create table from data, set -99 to null
table = Table.read(data_file, format='ascii')
for c in table.columns:
    for r in range(len(table)):
        if table[c][r] == -99.0: 
            table[c][r] = np.nan

In [4]:
# Fills in a new wavelength column in filter table
# wavelength in units of angstroms



filter_table = Table.read(filter_match, format='ascii.commented_header')
filter_table.add_column(np.zeros(1), name='central_wavelength')

for i in range(2, len(filter_table), 2): # starts at 3rd row to skip id and redshift, step = 2 to skip err
    # Open each transmission table in filters/, gather all transmission values, find max, then use that wavelength value.
    trans_table = Table.read(filters_dir + filter_table['CIGALE'][i] + '.dat', format='ascii')
    max_row = trans_table[0]
    for j in trans_table:
        if j['col2'] > max_row['col2']:
            max_row = j
    filter_table['central_wavelength'][i] = max_row['col1']

filter_table[0:]

ZFOURGE,CIGALE,central_wavelength
str9,str39,float64
id,id,0.0
z_peak,redshift,0.0
f_B,hst-ACS_update_sep07-wfc_f435w_t81,4565.0
e_B,hst-ACS_update_sep07-wfc_f435w_t81_err,0.0
f_I,hst-ACS_update_sep07-wfc_f775w_t81,7385.0
e_I,hst-ACS_update_sep07-wfc_f775w_t81_err,0.0
f_R,ESO-VIMOS-R,6230.0
e_R,ESO-VIMOS-R_err,0.0
f_U,ESO-vimos_u,3920.0


In [5]:
# Create ordered array of all wavelength values.
wavelengths = np.array(filter_table[2::2]['central_wavelength'], dtype='int')

# Create a 2d ordered array of fluxes for each galaxy
fluxes = np.empty((len(table),len(wavelengths)))
for i in range(len(filter_table[2::2])):
    fluxes[:,i] = table[filter_table['ZFOURGE'][(i+1)*2]] * 0.3631 / 1000 # Converts fluxes into units of mJy

# Create same array for errors
errors = np.empty((len(table),len(wavelengths)))
for i in range(len(filter_table[3::2])):
    errors[:,i] = table[filter_table['ZFOURGE'][(i+2)*2-1]] * 0.3631 / 1000 # Sanity check: correct error propogation?
    for j in range(len(errors[:,i])):
        if errors[j,i] < 0: errors[j,i] = np.nan # Sets negative errors to NaN 

In [6]:
# Plot!
for i in range(len(fluxes)):
    plt.errorbar(wavelengths, fluxes[i], yerr=errors[i], ecolor=(0.5,0.5,0.5), fmt='.', color=(1-(i/len(fluxes)),0,i/len(fluxes)))
    plt.xscale('log')
    plt.yscale('log')
    plt.axis((3000,90000,0.000001,3))
    plt.xlabel("Wavelength (angstroms)")
    plt.ylabel("Flux (mJy)")
    plt.title('ZFOURGE ID #' + str(table[i]['id']))
    plt.savefig(output_dir + 'ZFOURGE ID #' + str(table[i]['id']) + '.png')
    plt.close()

In [7]:
table[7]

id,x,y,ra,dec,SEflags,iso_area,fap_Ksall,eap_Ksall,apcorr,Ks_ratio,fapcirc07_Ksall,eapcirc07_Ksall,apcorr07,fcirc07_Ksall,ecirc07_Ksall,fauto_Ksall,flux50_radius,a_vector,b_vector,kron_radius,f_Ksall,e_Ksall,w_Ksall,f_B,e_B,w_B,f_I,e_I,w_I,f_R,e_R,w_R,f_U,e_U,w_U,f_V,e_V,w_V,f_Z,e_Z,w_Z,f_Hs,e_Hs,w_Hs,f_Hl,e_Hl,w_Hl,f_J1,e_J1,w_J1,f_J2,e_J2,w_J2,f_J3,e_J3,w_J3,f_Ks,e_Ks,w_Ks,f_KsHI,e_KsHI,w_KsHI,f_NB118,e_NB118,w_NB118,f_NB209,e_NB209,w_NB209,f_F098M,e_F098M,w_F098M,f_F105W,e_F105W,w_F105W,f_F125W,e_F125W,w_F125W,f_F140W,e_F140W,w_F140W,f_F160W,e_F160W,w_F160W,f_F814W,e_F814W,w_F814W,f_IA484,e_IA484,w_IA484,f_IA527,e_IA527,w_IA527,f_IA574,e_IA574,w_IA574,f_IA598,e_IA598,w_IA598,f_IA624,e_IA624,w_IA624,f_IA651,e_IA651,w_IA651,f_IA679,e_IA679,w_IA679,f_IA738,e_IA738,w_IA738,f_IA767,e_IA767,w_IA767,f_IA797,e_IA797,w_IA797,f_IA856,e_IA856,w_IA856,f_WFI_V,e_WFI_V,w_WFI_V,f_WFI_Rc,e_WFI_Rc,w_WFI_Rc,f_WFI_U38,e_WFI_U38,w_WFI_U38,f_tenisK,e_tenisK,w_tenisK,f_IRAC_36,e_IRAC_36,w_IRAC_36,f_IRAC_45,e_IRAC_45,w_IRAC_45,f_IRAC_58,e_IRAC_58,w_IRAC_58,f_IRAC_80,e_IRAC_80,w_IRAC_80,wmin_optical,wmin_hst_optical,wmin_fs,wmin_jhk,wmin_hst,wmin_irac,wmin_all,star,nearstar,use,snr,use_nosnr,z_spec
int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,int64,float64
378,2629.872,423.862,53.1684418,-27.9575253,0,15.0,0.7959871,0.251456,1.169054,1.979685,0.4787802,0.1291864,1.774505,0.8495979,0.229242,1.347931,2.89,1.341,0.862,4.913,1.5775373844000002,0.40951437084000003,0.11,nan,nan,0.0,nan,nan,0.0,-0.1129385048,0.0728612215,0.11,-0.001260270928,0.01524464872,0.93,nan,nan,0.0,nan,nan,0.0,nan,nan,0.0,nan,nan,0.0,nan,nan,0.0,nan,nan,0.0,nan,nan,0.0,nan,nan,0.0,nan,nan,0.0,nan,nan,0.0,nan,nan,0.0,nan,nan,0.0,nan,nan,0.0,nan,nan,0.0,nan,nan,0.0,nan,nan,0.0,0.0970302748,0.096370446,0.47,-0.05690815638,0.04734846704,1.0,-0.05159558124000001,0.04959841266,1.0,0.17311475999999998,0.12612073799999998,1.0,0.01068595341,0.04858215296000001,1.0,0.01229864911,0.05587887014,1.0,-0.019191927159999997,0.043226415799999994,1.0,0.03226157408,0.055498379679999994,1.0,-0.06689919456000001,0.06015448032,1.0,0.0018453471270000001,0.1508363649,1.0,-0.0327982577,0.17298699599999998,1.0,-0.4596941327,0.2308436084,1.0,0.03022552422,0.04112608093,1.15,-0.05509261935999999,0.037778485279999996,1.1,0.0012461866536,0.13971022379999998,0.96,1.7602447450000003,0.27977960080000003,0.95,2.2032858749999997,0.1518207565,0.1,1.5472677849999998,0.204109325,0.15,-195.9888,-195.9888,-0.0,-0.151152554,2.88472212,0.07,0.0,0.47,0.0,0.0,0.0,0.0,0.0,0,0,0,3.706118,0,nan


In [8]:
wavelengths

array([ 4565,  7385,  6230,  3920,  6685,  8605, 15525, 16725, 10800,
       11935, 13345, 21055, 22134, 11930, 21000, 10435, 11790, 13260,
       13335, 14445,  7145,  4905,  5250,  5845,  5985,  6220,  6520,
        6790,  7380,  7765,  7890,  8485,  5230,  6685,  3710, 21355,
       37490, 44500, 59840, 84900])

In [9]:
table[96]['e_IRAC_80'] # lol

30310.047599999998